<a href="https://colab.research.google.com/github/monzzzz/AIMO-Competition/blob/main/AIMO_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Download AIME Dataset**

In [ ]:
import pandas as pd
aime_problems_path = "/content/train.csv"
df = pd.read_csv(aime_problems_path)
problem_set = []
for index, line in df.iterrows():
  problem_set.append(line.problem)


In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install --upgrade transformers

# Deep Seek Math Model

In [ ]:
# Load model directly
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, set_seed)

set_seed(42)

# Qlora
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # load the model in 4 bit
    bnb_4bit_quant_type="nf4", # quantisize in the normalize float 4
    bnb_4bit_compute_dtype=torch.bfloat16, # type of the number used in the calculation
    bnb_4bit_use_double_quant=True, # quantisize 2 times
)

config = AutoConfig.from_pretrained(
    "deepseek-ai/deepseek-math-7b-instruct",
)
config.gradient_checkpointing = True

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-math-7b-instruct")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-math-7b-instruct", device_map="auto", torch_dtype="auto", config=config) # load with accelarate (device_map)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

model.generation_config

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 100000,
  "eos_token_id": 100001,
  "pad_token_id": 100001
}

In [ ]:
def predict(model, tokenizer, ps):
  ans = []
  for problem in ps:
    # messages = [{"role": "user", "content": problem + "please provide your answer within \\boxed{}"}]
    input = tokenizer(problem, return_tensors="pt") # retunrn tensor in the pytorch format
    # generate_id = model.generate(input.to(model.device), max_new_tokens=200, pad_token_id = tokenizer.eos_token_id)
    # output = tokenizer.batch_decode(generate_id, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    output = model.generate(**input.to(model.device), max_new_tokens = 100)
    ans.append(output)
  return ans

In [ ]:
ans = predict(model, tokenizer, problem_set)

In [ ]:
import re
answer_list = []
for index, sans in enumerate(ans):
  # print(f"problem {index + 1}" + "\n")
  # print(sans[0])
  # matching = re.search(r'\{(.+?)\}', sans[0])
  answer_list.append(sans[0])

for answer_nums,answer in enumerate(answer_list):
  result = tokenizer.decode(answer, skip_special_tokens=True)
  index = 0;
  target = "boxed{"
  nums_answer = ""
  for string_index, i in enumerate(result):
    if (i == target[index]):
      index+=1
      if (index == 5):
        nums_index = 2
        while (result[string_index + nums_index] != '}'):
          nums_answer  = nums_answer + result[string_index + nums_index]
          nums_index+=1
        index = 0
    else:
      index= 0
  if (nums_answer == ""):
    print(f"problem {answer_nums+1}: " + "no answer")
  else:
    print(f"problem {answer_nums+1}: " + nums_answer)
# for index, answer in enumerate(answer_list):
#   print(f"problem {index+1}" + "\n")
#   print(answer + "\n\n")


problem 1: no answer
problem 2: 8
problem 3: no answer
problem 4: no answer
problem 5: 1155
problem 6: B
problem 7: no answer
problem 8: a + b + c + da+b+c+d
problem 9: no answer
problem 10: no answer
